In [75]:
import json

In [83]:
file="tailored_examples-test.jsonl"

In [84]:
with open(file) as f:
	data = [json.loads(line)["@graph"] for line in f]

print(data)

[[{'@id': 'https://libris-qa.kb.se/test/text', '@type': 'Record', 'mainEntity': {'@id': 'https://libris-qa.kb.se/test/text#it'}}, {'@id': 'https://libris-qa.kb.se/test/text#it', '@type': 'PhysicalResource', 'instanceOf': {'@type': 'Monograph', 'category': 'https://id.kb.se/term/rda/Text', 'hasTitle': [{'@type': 'Title', 'mainTitle': 'Verkstyp Text blir RDA Text'}]}}], [{'@id': 'https://libris-qa.kb.se/test/work', '@type': 'Record', 'mainEntity': {'@id': 'https://libris-qa.kb.se/test/work#it'}}, {'@id': 'https://libris-qa.kb.se/test/work#it', '@type': 'Print', 'instanceOf': {'@type': 'Work', 'hasTitle': [{'@type': 'Title', 'mainTitle': 'Verkstyp Work sparas inte som kategori'}]}, 'issuanceType': 'Monograph'}]]


In [ ]:
for r in data:
	id = r[0].get("@id", "")
	record = r[1]

	try:
		title = record["instanceOf"]["hasTitle"][0]["mainTitle"]
	except:
	   title = {record["hasTitle"][0]["mainTitle"]}

	work_type = record.get("instanceOf", {}).get("@type", "")
	work_category = record.get("instanceOf", {}).get("category", [])
	work_genreforms = record.get("instanceOf", {}).get("genreForm", [])
	work_contentType = record.get("contentType", [])

	
	instance_type = record.get("@type", "")
	instance_category = record.get("category", [])
	instance_genreforms = record.get("genreForm", [])
	instance_carriertype = record.get("carrierType", [])
	instance_mediatype = record.get("mediaType", [])



	print("\n#", title)

	print(f'elif id == "{id}":')
	print('''\tassert work_type == ""
	assert work_category == []
	
	

	assert instance_type == ""
	assert instance_category == []
	
	
	''')


## Kör tester

In [92]:
for r in data:
    id = r[0].get("@id", "")
    record = r[1]

    try:
        title = record["instanceOf"]["hasTitle"][0]["mainTitle"]
    except:
        # Special case
        title = {record["hasTitle"][0]["mainTitle"]}

    print(f"\n{id}\t{title}")

    work_type = record.get("instanceOf", {}).get("@type", "")
    work_category = record.get("instanceOf", {}).get("category", [])

    instance_type = record.get("@type", "")
    instance_category = record.get("category", [])

    """Genreic test for leftover obsolete properties"""

    old_properties = [
        "genreForm",
        "contentType",
        "carrierType",
        "mediaType",
        "issuanceType",
    ]

    for p in old_properties:
        assert p not in record.get(
            "instanceOf", {}
        ), "Failed! Obsolete property still present in work: {p}"
        assert (
            p not in record
        ), "Failed! Obsolete property still present in instance: {p}"

    """Specific tests for each tailored example"""

    # Verkstyp Text blir RDA Text
    if id == "https://libris-qa.kb.se/test/text":
        assert work_category == ["https://id.kb.se/term/rda/Text"], record

    # Verkstyp Work sparas inte som kategori
    elif id == "https://libris-qa.kb.se/test/work":
        assert work_type == "Work", record

    # Verkstyp MovingImage blir ktg MovingImage
    elif id == "https://libris-qa.kb.se/test/movingimage":
        assert work_category == ["https://id.kb.se/term/ktg/MovingImage"], record

    # Verkstyp Audio blir ktg Audio
    elif id == "https://libris-qa.kb.se/test/audio":
        assert work_category == ["https://id.kb.se/term/ktg/Audio"], record

    # Verkstyp NotatedMusic blir RDA NotatedMusic
    elif id == "https://libris-qa.kb.se/test/notatedmusic":
        assert work_category == ["https://id.kb.se/term/rda/NotatedMusic"], record

    # Verkstyp StillImage blir RDA StillImage
    elif id == "https://libris-qa.kb.se/test/stillimage":
        assert work_category == ["https://id.kb.se/term/ktg/StillImage"], record

    # Verkstyp Music blir ktg Music
    elif id == "https://libris-qa.kb.se/test/music":
        assert work_category == ["https://id.kb.se/term/ktg/Music"], record

    # Verkstyp Multimedia blir ktg Multimedia
    elif id == "https://libris-qa.kb.se/test/multimedia":
        assert work_category == ["https://id.kb.se/term/ktg/Multimedia"], record

    # Verkstyp Cartography blir RDA CartographicImage om ej annan RDA Cartorgaphy-term redan finns
    elif id == "https://libris-qa.kb.se/test/cartography":
        assert work_category == ["htps://id.kb.se/term/ktg/CartographicImage"], record

    # Verkstyp Kit blir ktg Kit
    elif id == "https://libris-qa.kb.se/test/kit":
        assert work_category == ["https://id.kb.se/term/ktg/Kit"], record

    # Verkstyp Object blir RDA ThreeDimensionalForm
    elif id == "https://libris-qa.kb.se/test/object":
        assert work_category == [
            "https://id.kb.se/term/rda/ThreeDimensionalForm"
        ], record

    # Verkstyp ManuscriptText blir SAOGF Handsrkrifter
    elif id == "https://libris-qa.kb.se/test/manuscripttext":
        assert work_category == ["https://id.kb.se/term/saogf/Handskrifter"], record

    # Verkstyp ManuscriptNotatedMusic blir RDA NotatedMusic och SAOGF Handskrifter
    elif id == "https://libris-qa.kb.se/test/manuscriptnotatedmusic":
        assert work_category == [
            "https://id.kb.se/term/rda/NotatedMusic",
            "https://id.kb.se/term/saogf/Handskrifter",
        ], record

    # Verkstyp MixedMaterial blir ktg MixedMaterial
    elif id == "https://libris-qa.kb.se/test/mixedmaterial":
        assert work_category == ["https://id.kb.se/term/ktg/MixedMaterial"], record

    # Verkstyp Dataset blir ktg Dataset
    elif id == "https://libris-qa.kb.se/test/dataset":
        assert work_category == ["https://id.kb.se/term/ktg/Dataset"], record

    # Verkstyp NonMusicAudio blir RDA Sounds
    elif id == "https://libris-qa.kb.se/test/nonmusicaudio":
        assert work_category == ["https://id.kb.se/term/rda/Sounds"], record

    # Verkstyp saknas i mappning - skriv ut i logg!
    elif id == "https://libris-qa.kb.se/test/unhandledworktype":
        assert work_type == "Work", record

    # Instanstyp Instance och verkstyp Text -> instanstyp PhysicalResource och verkskategori Text
    elif id == "https://libris-qa.kb.se/test/instance":
        assert work_type == "Text", record
        assert instance_type == "PhysicalResource", record

    # Instanstyp Electronic, inget som antyder Online -> Instanstyp PhysicalResource, instanskategori ElectronicStorageMedium
    elif id == "https://libris-qa.kb.se/test/electronic":
        assert instance_type == "PhysicalResource", record
        assert instance_category == [
            "https://id.kb.se/term/ktg/ElectronicStorageMedium"
        ], record

    # Instanstyp Print blir ktg Print
    elif id == "https://libris-qa.kb.se/test/print":
        assert instance_category == ["https://id.kb.se/term/ktg/Print"]

    # Instanstyp VideoRecording, inget som antyder Online -> Instanstyp PhysicalResource, instanskategori ktg/VideoStorageMedium, verkskategori ktg/MovingImage
    elif id == "https://libris-qa.kb.se/test/videorecording":
        assert work_category == ["https://id.kb.se/term/ktg/MovingImage"], record
        assert instance_category == [
            "https://id.kb.se/term/ktg/VideoStorageMedium"
        ], record

    # Instanstyp SoundRecording, inget som antyder Online -> Instanstyp PhysicalResource, instanskategori ktg/SoundStorageMedium, verkskategori ktg/Audio
    elif id == "https://libris-qa.kb.se/test/soundrecording":
        assert work_category == ["https://id.kb.se/term/ktg/Audio"], record
        assert instance_type == "PhysicalResource"
        assert instance_category == [
            "https://id.kb.se/term/ktg/SoundStorageMedium"
        ], record

    # Instanstyp StillImageInstance -> instanstyp PhysicalResource, instanskategori rda/Sheet och verkskateogri rda/StillImage
    elif id == "https://libris-qa.kb.se/test/stillimageinstance":
        assert work_category == ["https://id.kb.se/term/rda/StillImage"], record
        assert instance_type == "PhysicalResource", record
        assert instance_category == ["https://id.kb.se/term/rda/Sheet"], record

    # Instanstyp Tactile -> instanstyp PhysicalResource, ktg Tactile
    elif id == "https://libris-qa.kb.se/test/tactile":
        assert work_type == ""
        assert work_category == []
        assert instance_type == ""
        assert instance_category == []

    # Instanstyp Map -> instanstyp PhysicalResource, och verkskategori rda/CartographicImage
    elif id == "https://libris-qa.kb.se/test/map":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Instanstyp Manuscript -> instanstyp PhysicalResource, instanskategori ktg/ManuscriptForm
    elif id == "https://libris-qa.kb.se/test/manuscript":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Instanstyp Microform -> instanstyp PhysicalResource, instanskategori rda/Microform
    elif id == "https://libris-qa.kb.se/test/microform":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Instanstyp Globe -> instanstyp PhysicalResource, instanskategori rda/CartographicThreeDimensionalForm, verkskategori ktg/Globe och rda/CartographicImage
    elif id == "https://libris-qa.kb.se/test/globe":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Instanstyp KitInstance med verkstyp Text
    elif id == "https://libris-qa.kb.se/test/kitinstance":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Instanstyp TextInstance -> instanskategori ktg/Volume och verkskategori rda/Text
    elif id == "https://libris-qa.kb.se/test/textinstance":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Instanstyp Archival med verkstyp Text
    elif id == "https://libris-qa.kb.se/test/archival":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # issuanceType ComponentPart blir instans-ktg ComponentPart, verkstyp Monograph
    elif id == "https://libris-qa.kb.se/test/componentPart":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # issuanceType ComponentPart blir instans-ktg ComponentPart, verkstyp Monograph
    elif id == "https://libris-qa.kb.se/test/componentPartInstance":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # issuanceType serialComponentPart blir instans-ktg ComponentPart, verkstyp Monograph
    elif id == "https://libris-qa.kb.se/test/serialComponentPart":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Städa bort den tvetydiga MARC-typen Other från Kategori på instans
    elif id == "https://libris-qa.kb.se/test/marcother":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Flytta länkade termer i genreForm på instansen till kategori på verket efter att instansen normaliserats. Reducera och mappa verkskategorier som vanligt.
    elif id == "https://libris-qa.kb.se/test/moveInstanceGenreForm":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Flytta länkade termer i genreForm på instansen till kategori på verket efter att instansen normaliserats. Reducera och mappa verkskategorier som vanligt.
    elif id == "https://libris-qa.kb.se/test/moveInstanceGenreFormNoInstanceCategory":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # På verk: gammal hasPart Text blir Work
    elif id == "https://libris-qa.kb.se/test/workHasPartText":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # På verk: gammal hasPart Print blir PhysicalResource
    elif id == "https://libris-qa.kb.se/test/workHasPartPrint":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # På instans: Om gammal hasPart är Electronic och huvudpostens nya typ DigitalResource, blir hasPart DigitalResources
    elif id == "https://libris-qa.kb.se/test/instanceHasPartElectronic":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # På instans: gammal hasPart Print blir PhysicalResource; gammal hasPart Text blir Work
    elif id == "https://libris-qa.kb.se/test/instanceHasPartsTextPrint":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # {'Specialfall Signe: verk som inte länkas från instanser blir också normaliserade'}
    elif id == "https://libris-qa.kb.se/test/workNotLinkedFromInstance":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # {'Specialfall Signe: om det inte finns en issuanceType på instansen, hämta den från verket'}
    elif id == "https://libris-qa.kb.se/test/workWithIssuanceType":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Ta bort bredare kategori 'Kataloger' som impliceras av smalare 'Bibliotekskataloger'
    elif id == "https://libris-qa.kb.se/test/removeImpliedBroader":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Implicera aldrig contentType - alla bevaras explicit
    elif id == "https://libris-qa.kb.se/test/neverImplyContentType":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # FAIL? TYDLIGGÖR VÄNTAT RESULTAT Om instanstyp är Globe och verkstyp Cartpgraphy, behandla verket som CartographicObject
    elif id == "https://libris-qa.kb.se/test/complexInstanceTypeMappingGlobe":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Om instanstyp är Map och verkstyp Cartography, behandla verket som CartographicImage
    elif id == "https://libris-qa.kb.se/test/complexInstanceTypeMappingMap":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Omappad instanstyp MovingImageInstance -> fallback PhysicalResource
    elif id == "https://libris-qa.kb.se/test/unmappedInstanceTypeMovingImageInstance":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # När instanstyp är Electronic och minst en carrierTypes innehåller Online - sätt instanstyp DigitalResource
    elif id == "https://libris-qa.kb.se/test/digitalResourceFromElectronic":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # När instanstyp är Instance och carrierTypes innehåller termer som innehåller/är mappade till 'Online' och 'Electronic' - sätt instanstyp DigitalResource
    elif id == "https://libris-qa.kb.se/test/digitalResourceFromInstance":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # DENNA SKA FAILA - SE KOMMENTAR I KOD När instanstyp är Electronic och carrierTypes innehåller Online - ta bort länkade carrierTypes med Online i URIn, och om inga and carrierTypes finns kvar efetr det lägg på OnlineResource
    elif id == "https://libris-qa.kb.se/test/cleanUpOnlineCategories":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Sätt instanskategori rda/volume om extent innehåller uppgift om sidor
    elif id == "https://libris-qa.kb.se/test/computeVolumeFromExtent":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Byr ut lokal entitet 'E-böcker' mot ktg/E-books.
    elif id == "https://libris-qa.kb.se/test/replaceLocalEböckerWithEbooks":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Bevara nya typer Monograph och DigitalResource på redan normaliserade poster
    elif id == "https://libris-qa.kb.se/test/alreadyNormalized":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Städa bärartyp utan att ta bort RDA-termer
    elif id == "https://libris-qa.kb.se/test/multipleCarrierTypesOneOnline":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Videoinspelningar ska inte ktg/Print, även om de har egenskapen publication
    elif id == "https://libris-qa.kb.se/test/doNotAddPrintOnVideoWhenPublication":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []

    # Ljudinspelningar ska inte ktg/Print, även om de har egenskapen publication
    elif id == "https://libris-qa.kb.se/test/doNotAddPrintOnSoundWhenPublication":
        assert work_type == ""
        assert work_category == []

        assert instance_type == ""
        assert instance_category == []


https://libris-qa.kb.se/test/text	Verkstyp Text blir RDA Text


AssertionError: {'@id': 'https://libris-qa.kb.se/test/text#it', '@type': 'PhysicalResource', 'instanceOf': {'@type': 'Monograph', 'category': 'https://id.kb.se/term/rda/Text', 'hasTitle': [{'@type': 'Title', 'mainTitle': 'Verkstyp Text blir RDA Text'}]}}